<a href="https://colab.research.google.com/github/Annesya/voice-speech-metamers/blob/dev_branch/embedding_extraction_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, AutoFeatureExtractor, WhisperModel

getting embeddings from whisper: https://huggingface.co/docs/transformers/model_doc/whisper#transformers.WhisperModel.forward.output_hidden_states

tutorial on using whisper with common voice: https://huggingface.co/blog/fine-tune-whisper

getting ECAPA-TDNN emveddings: https://huggingface.co/speechbrain/spkrec-ecapa-voxceleb

In [3]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
from collections import defaultdict

import torch
from transformers import Wav2Vec2Model
from speechbrain.inference.encoders import WaveformEncoder

In [7]:
%%capture
!pip install datasets -U
!pip install librosa
!pip install jiwer

In [5]:
class Encoder(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
        if self.model_name == "wav2vec2_orig":
            self.model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        else:
            self.model = WaveformEncoder.from_hparams(source=ckpt, savedir=cache_dir, run_opts={"device":"cuda"})
            self.activation = defaultdict(list)
            for layer in range(12):
                getattr(self.model.mods.encoder.encoder_wrapper.latent_encoder.layers, str(layer)).register_forward_hook(self.get_layer_embeddings(f'encoder_features_{layer}'))

    def get_layer_embeddings(self, name):
        def hook(model, input, output):
            self.activation[name].append(output[0])
        return hook

    def forward(self, input_values):
        device = input_values[0].device
        if self.model_name == "wav2vec2_orig":
            output_values = [self.model(input_.squeeze(0), output_hidden_states=True) for input_ in input_values]
            return [list(output_.hidden_states[1:]) for output_ in output_values]
        else:
            inputs_len = torch.FloatTensor([len(input_) for input_ in input_values]).to(device)
            inputs_len_norm = inputs_len/inputs_len.max()
            _ = [self.model.encode_batch(input_.squeeze(0), inputs_len_norm[i])["embeddings"].squeeze(0) for i, input_ in enumerate(input_values)]
            # return self.activation
            output_values = []
            for i in range(len(input_values)):
                output_values.append([self.activation[name][i] for name in self.activation.keys()])
            self.activation = defaultdict(list)
            return output_values


if __name__ == '__main__':
    model = Encoder("wav2vec2_orig")
    # model = Encoder("wav2vec2_orig", "facebook/wav2vec2-base", "/om2/user/gelbanna/model_cache")
    # x = {}
    x = [torch.ones((1,1,35000)), torch.ones((1,1,30000))]
    print(x)
    # compute the forward pass
    y = model(x)
    # print(y.keys(), len(y["encoder_features_0"]), y["encoder_features_0"][1].shape)
    print(len(y))
    print(len(y[0]), len(y[1]))
    print(y[0][0].shape, y[1][0].shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[tensor([[[1., 1., 1.,  ..., 1., 1., 1.]]]), tensor([[[1., 1., 1.,  ..., 1., 1., 1.]]])]
2
12 12
torch.Size([1, 109, 768]) torch.Size([1, 93, 768])


In [9]:
## generating the commonvoice dataset

from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice_train = load_dataset("fsicoli/common_voice_17_0", "en", split="train")
common_voice_test = load_dataset("fsicoli/common_voice_17_0", "en", split="test")

print(common_voice)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for fsicoli/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fsicoli/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6353it [00:00, 63504.02it/s]
Reading metadata...: 12704it [00:00, 60360.28it/s]
Reading metadata...: 18751it [00:00, 57858.02it/s]
Reading metadata...: 24549it [00:00, 57375.85it/s]
Reading metadata...: 30293it [00:00, 56503.40it/s]
Reading metadata...: 36755it [00:00, 59167.17it/s]
Reading metadata...: 43042it [00:00, 60348.31it/s]
Reading metadata...: 49129it [00:00, 60507.48it/s]
Reading metadata...: 55187it [00:00, 58955.19it/s]
Reading metadata...: 61095it [00:01, 57302.00it/s]
Reading metadata...: 66919it [00:01, 57575.37it/s]
Reading metadata...: 73055it [00:01, 58694.64it/s]
Reading metadata...: 78936it [00:01, 56754.87it/s]
Reading metadata...: 87429it [00:01, 64951.01it/s]
Reading metadata...: 97235it [00:01, 74670.78it/s]
Reading metadata...: 107463it [00:01, 82826.08it/s]
Reading metadata...: 117008it [00:01, 86570.39it/s]
Reading metadata...: 127379it [00:01, 91671.95it/s]
Reading metadata...: 137166it [00:01, 9

Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6215it [00:00, 62139.10it/s]
Reading metadata...: 16393it [00:00, 57727.66it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6520it [00:00, 65191.51it/s]
Reading metadata...: 16726it [00:00, 86873.24it/s]
Reading metadata...: 26988it [00:00, 94058.56it/s]
Reading metadata...: 37785it [00:00, 99251.87it/s]
Reading metadata...: 47708it [00:00, 95513.11it/s]
Reading metadata...: 58352it [00:00, 99126.18it/s]
Reading metadata...: 68513it [00:00, 99917.99it/s]
Reading metadata...: 79005it [00:00, 101488.17it/s]
Reading metadata...: 89858it [00:00, 103667.80it/s]
Reading metadata...: 101240it [00:01, 106779.74it/s]
Reading metadata...: 111928it [00:01, 106601.18it/s]
Reading metadata...: 123530it [00:01, 109449.82it/s]
Reading metadata...: 134481it [00:01, 109045.81it/s]
Reading metadata...: 145390it [00:01, 108354.47it/s]
Reading metadata...: 156230it [00:01, 102312.00it/s]
Reading metadata...: 166530it [00:01, 101364.83it/s]
Reading metadata...: 176713it [00:01, 99555.53it/s] 
Reading metadata...: 186944it [00:01, 100343.94it/s]
Reading metadata...: 1

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 5629it [00:00, 56282.67it/s]
Reading metadata...: 14093it [00:00, 72958.61it/s]
Reading metadata...: 25260it [00:00, 90628.82it/s]
Reading metadata...: 36198it [00:00, 98025.93it/s]
Reading metadata...: 47250it [00:00, 102527.80it/s]
Reading metadata...: 58127it [00:00, 104645.71it/s]
Reading metadata...: 69025it [00:00, 106059.97it/s]
Reading metadata...: 79632it [00:00, 104605.96it/s]
Reading metadata...: 90097it [00:00, 103680.79it/s]
Reading metadata...: 100469it [00:01, 103440.97it/s]
Reading metadata...: 110816it [00:01, 102587.40it/s]
Reading metadata...: 121078it [00:01, 99128.24it/s] 
Reading metadata...: 131559it [00:01, 100789.72it/s]
Reading metadata...: 141660it [00:01, 98910.87it/s] 
Reading metadata...: 151747it [00:01, 99481.58it/s]
Reading metadata...: 162036it [00:01, 100483.32it/s]
Reading metadata...: 172097it [00:01, 99429.11it/s] 
Reading metadata...: 182050it [00:01, 97667.61it/s]
Reading metadata...: 

DatasetDict({
    
})


In [12]:
common_voice_train[0]["audio"]["array"]

array([ 0.00000000e+00, -9.21357039e-14, -1.42753207e-14, ...,
       -6.54685550e-07, -1.42374017e-06, -4.96159998e-07])

In [13]:
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

inputs = feature_extractor(common_voice_train[0]["audio"]["array"], return_tensors="pt")
input_features = inputs.input_features
decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state
list(last_hidden_state.shape)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[1, 2, 512]

In [14]:
common_voice_train[0]

{'client_id': 'f9f1f96bae1390dfe61ff298abb90975c079e913c712d57d97307ed797469eac446abb149daaad24cacffcc24e1e3275fefeb97f977eb74ce2233e0e5c1d437e',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/224bf374e1809b99bfdc5af9e8502f280455bc2686cd93832b694a030955252f/en_train_0/common_voice_en_100229.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/224bf374e1809b99bfdc5af9e8502f280455bc2686cd93832b694a030955252f/en_train_0/common_voice_en_100229.mp3',
  'array': array([ 0.00000000e+00, -9.21357039e-14, -1.42753207e-14, ...,
         -6.54685550e-07, -1.42374017e-06, -4.96159998e-07]),
  'sampling_rate': 48000},
 'sentence': "I'm getting them for twelve dollars a night.",
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'en',
 'segment': '',
 'variant': ''}

In [17]:
from speechbrain.inference.speaker import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
signal = common_voice_train[0]["audio"]["array"]
# fs = common_voice_train[0]["audio"]["sampling_rate"]
embeddings = classifier.encode_batch(torch.tensor(signal))
list(embeddings.shape)

[1, 1, 192]